# LIMPIEZA DE SECOP I PARA LAS IPS PUBLICAS
Este segundo codigo busca hacer una limpieza en la base de datos SECOP I de las IPS públicas

In [1]:
# Cargar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Cargar datos
secopIps = pd.read_csv('Datos/secop_i_ips.csv.gz', sep=";")

In [3]:
secopIps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1640241 entries, 0 to 1640240
Data columns (total 44 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   anno_cargue                       1640241 non-null  int64  
 1   anno_firma                        1466790 non-null  float64
 2   nivel_entidad                     1640241 non-null  object 
 3   orden_entidad                     1640241 non-null  object 
 4   nombre_entidad                    1640241 non-null  object 
 5   nit_entidad                       1640241 non-null  object 
 6   tipo_proceso                      1640241 non-null  object 
 7   estado_proceso                    1640241 non-null  object 
 8   causa_otras_formas_contr_directa  1640241 non-null  object 
 9   regimen_contratacion              1640241 non-null  object 
 10  objeto_contratar                  1640241 non-null  object 
 11  detalle_objeto_contratar          163

***
### 1. Revisar los contratos con datos faltantes en el anno_firma

Inicialmente se tienen 173,451 contratos con datos faltantes en anno_firma

In [4]:
secopIpsNa = secopIps.loc[secopIps.anno_firma.isnull()]
secopIpsNa.head()

,anno_cargue,anno_firma,nivel_entidad,orden_entidad,nombre_entidad,nit_entidad,tipo_proceso,estado_proceso,causa_otras_formas_contr_directa,regimen_contratacion,...,fecha_fin_ejec_contrato,cuantia_contrato,valor_adiciones,valor_total_con_adiciones,objeto_contrato_a_la_firma,moneda,ruta_web,municipio_entidad,departamento_entidad,nombre_entidad_mod
10,2016,NaN,TERRITORIAL,DISTRITO CAPITAL,BOGOTÁ D.C. - HOSPITAL EL TUNAL,800209488-1,Contratación Directa (Ley 1150 de 2007),Convocado,Prestación de Servicios Profesionales y de Apo...,Estatuto General de Contratación,...,NaN,0,0,0,No definido,No Definida,https://www.contratos.gov.co/consultas/detalle...,Bogotá D.C.,Bogotá D.C.,bogotá dc hospital el tunal
14,2016,NaN,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,LA GUAJIRA - HOSPITAL SANTO TOMAS DE VILLANUEVA,800075650,Régimen Especial,Convocado,Ninguna,Régimen Especial,...,NaN,0,0,0,No definido,No Definida,https://www.contratos.gov.co/consultas/detalle...,Villanueva,La Guajira,la guajira hospital santo tomas de villanueva
16,2017,NaN,TERRITORIAL,TERRITORIAL DEPARTAMENTAL CENTRALIZADO,ATLÁNTICO - E.S.E. HOSPITAL DEPARTAMENTAL DE S...,890103127,Contratación Directa (Ley 1150 de 2007),Convocado,Prestación de Servicios Profesionales y de Apo...,Estatuto General de Contratación,...,NaN,0,0,0,No definido,No Definida,https://www.contratos.gov.co/consultas/detalle...,Sabanalarga,Atlántico,atlántico ese hospital departamental de saban...
19,2018,NaN,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,MAGDALENA - E.S.E HOSPITAL FRAY LUIS DE LEÓN -...,819001483,Régimen Especial,Convocado,Ninguna,Régimen Especial,...,NaN,0,0,0,No definido,No Definida,https://www.contratos.gov.co/consultas/detalle...,Plato,Magdalena,magdalena ese hospital fray luis de león plato
45,2016,NaN,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,VALLE DEL CAUCA - E.S.E. RED DE SALUD DEL ORIE...,805027337-4,Régimen Especial,Convocado,Ninguna,Régimen Especial,...,NaN,0,0,0,No definido,No Definida,https://www.contratos.gov.co/consultas/detalle...,Cali,Valle del Cauca,valle del cauca ese red de salud del oriente ...


Despues de verificar varias las columnas estado del proceso, tipo id del contratista, id del contratista y cuantia del contrato se encontro que no hay información suficiente para considerar estos contratos. Por tanto se filtra la base de datos quitando estos contratos

In [5]:
# Eliminar contratos 
secopIps.dropna(subset=['anno_firma'], inplace=True)
secopIps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1466790 entries, 0 to 1640240
Data columns (total 44 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   anno_cargue                       1466790 non-null  int64  
 1   anno_firma                        1466790 non-null  float64
 2   nivel_entidad                     1466790 non-null  object 
 3   orden_entidad                     1466790 non-null  object 
 4   nombre_entidad                    1466790 non-null  object 
 5   nit_entidad                       1466790 non-null  object 
 6   tipo_proceso                      1466790 non-null  object 
 7   estado_proceso                    1466790 non-null  object 
 8   causa_otras_formas_contr_directa  1466790 non-null  object 
 9   regimen_contratacion              1466790 non-null  object 
 10  objeto_contratar                  1466790 non-null  object 
 11  detalle_objeto_contratar          146

***
### 2. Analizar las fecha de inicio y finalización de contratos

In [6]:
dateCol = ['fecha_fin_ejec_contrato', 'fecha_inicio_ejec_contrato', 'fecha_firma_contrato']
for i in dateCol:
    secopIps[i] = pd.to_datetime(secopIps[i], format = '%m/%d/%Y', dayfirst=True)

In [7]:
secopIps.fecha_fin_ejec_contrato.head()

0   2015-06-01
1   2018-11-16
2   2018-01-01
3   2016-12-01
4   2016-12-01
Name: fecha_fin_ejec_contrato, dtype: datetime64[ns]

In [8]:
temp = secopIps.loc[(secopIps.fecha_fin_ejec_contrato < secopIps.fecha_firma_contrato) |
                    (secopIps.fecha_inicio_ejec_contrato < secopIps.fecha_firma_contrato)]

In [9]:
print(f'Existen {temp.shape[0]} contratos donde la fechas de inicio y fin de ejecución son superiores a las fechas de firma.')

Existen 14200 contratos donde la fechas de inicio y fin de ejecución son superiores a las fechas de firma.


In [10]:
secopIps.fecha_firma_contrato.min(), secopIps.fecha_inicio_ejec_contrato.min(), secopIps.fecha_fin_ejec_contrato.min(), 

(Timestamp('2011-01-01 00:00:00'),
 Timestamp('1990-05-08 00:00:00'),
 Timestamp('1990-12-08 00:00:00'))

> Revisar contratos que con fecha de inicio y ejecución con fechas de 1990.
> El problema de estos contratos es que la fecha de inicio y fin estan por debajo de la de la fecha de firma. Por tanto, se deben ajustar manualmente.

En este caso primero filtraremos primero los contratos con año de firma a partir del 2014

In [11]:
secopIps = secopIps.loc[secopIps.anno_firma >= 2014]
print(f'Despues de filtrar los contratos con fecha de firma a partir de 2014 se tienen {secopIps.shape[0]} contratos .')

Despues de filtrar los contratos con fecha de firma a partir de 2014 se tienen 1424873 contratos .


***
### 3. Retirar contratos de Convenios interadministativos


In [12]:
temp = secopIps.loc[secopIps.detalle_objeto_contratar.str.lower().str.contains('aunar esfuerzos|anuar esfuerzos|aunar espuerzos|convenio interadministrativo|conveio interaministrativo', 
                                                                    regex=True, na=False)]
print(f'Se tienen {temp.shape[0]} contratos de convenios interadministrativos')

Se tienen 12257 contratos de convenios interadministrativos


In [13]:
secopIps = secopIps.loc[-secopIps.detalle_objeto_contratar.str.lower().str.contains('aunar esfuerzos|anuar esfuerzos|aunar espuerzos|convenio interadministrativo|conveio interaministrativo', 
                                                                    regex=True, na=False)]
print(f'Despues de eliminar los contratos interadministrativos se tienen {secopIps.shape[0]} contratos.')

Despues de eliminar los contratos interadministrativos se tienen 1412616 contratos.


***
### 4. Contratos asociados a transacciones bancarias


In [14]:
transBank = ['Crédito','Fiducia','Comodato','Arrendamiento']
secopIps = secopIps.loc[-secopIps.tipo_contrato.isin(transBank)]

In [15]:
print(f'Despues de eliminar los contratos con transacciones bancarias se tienen {secopIps.shape[0]} contratos.')

Despues de eliminar los contratos con transacciones bancarias se tienen 1405302 contratos.


***
### 5. Analizar registros donde año de cargue > año de firma

Se debe indagar un poco mas sobre el porque es necesario hace esta depuración. Ya que revisando algunos contratos filtrados no se encuentra mayor problema

In [16]:
temp = secopIps.loc[secopIps.anno_cargue > secopIps.anno_firma]
print(f'Se tienen {temp.shape[0]} contratos donde año de cargue esta despues del año de firma.')

Se tienen 147278 contratos donde año de cargue esta despues del año de firma.


***
### 6. Buscar y corregir posibles errores por cuantia o plazo de ejecución

Queda pendiente corregir manualmente valores altos y bajos en las cuantas de los contratos y las adiciones. Asi, como los plazos de ejecución.

***
### 7. Quitar entidades que no corresponden a hospitales


In [17]:
ent_eliminar = ["CÁMARA DE REPRESENTANTES",
                "CDMB - CORPORACIÓN AUTÓNOMA REGIONAL PARA LA DEFENSA DE LA MESETA DE BUCARAMANGA",
                "VALLE DEL CAUCA - INSTITUTO PARA LA INVESTIGACIÓN Y LA PRESERVACIÓN DEL PATRIMONIO CULTURAL Y NATURAL DEL VALLE - CALI",
                "META - ALCALDÍA MUNICIPIO DE MESETAS",
                "INSTITUTO DE PLANIFICACIÓN Y PROMOCIÓN DE SOLUCIONES ENERGÉTICAS PARA LAS ZONAS NO INTERCONECTADAS (IPSE)",
                "TOLIMA - INSTITUCIÓN EDUCATIVA EXALUMNAS DE LA PRESENTACIÓN - IBAGUÉ",
                "NARIÑO - EMPRESA DE SERVICIOS PÚBLICOS DOMICILIARIOS DE PUERRES ESERP E.S.P.",
                "META - INSTITUCIÓN  EDUCATIVA  LOS  FUNDADORES - MESETAS",
                "ANTIOQUIA - INSTITUCIÓN EDUCATIVA LA PRESENTACIÓN - MEDELLÍN",
                "CALDAS - I.E. ESCUELA NORMAL SUPERIOR DE LA PRESENTACIÓN - PENSILVANIA",
                "VALLE DEL CAUCA - INSTITUTO MUNICIPAL PARA EL DESARROLLO SOCIAL Y ECONOMICO - IMDESEPAL",
                "CASANARE - INSTITUTO TÉCNICO EDUCATIVO LA PRESENTACIÓN MUNICIPIO DE SAN LUIS DE PALENQUE",
                "CALDAS - INSTITUCIÓN EDUCATIVA LA PRESENTACIÓN - SALAMINA",
                "SUCRE - INSTITUCIÓN EDUCATIVA ISMAEL CONTRERAS MENESES - COVEÑAS",
                "RISARALDA - INSTITUCIÓN EDUCATIVA NUESTRA SEÑORA DE LA PRESENTACIÓN - LA VIRGINIA",
                "BOYACÁ - ESCUELA NORMAL SUPERIOR LA PRESENTACIÓN - SOATÁ",
                "SANTANDER - COLEGIO TÉCNICO NUESTRA SEÑORA DE LA PRESENTACIÓN - SAN GIL",
                "NARIÑO - INSTITUCIÓN EDUCATIVA TELESECUNDARIA DE SAN GERARDO - LA CRUZ",
                "META - CONCEJO MUNICIPIO DE MESETAS",
                "CASANARE - INSTITUCIÓN EDUCATIVA CENTRO SOCIAL LA PRESENTACIÓN - YOPAL",
                "NORTE DE SANTANDER - INSTITUCIÓN EDUCATIVA TÉCNICA NUESTRA SEÑORA DE LA PRESENTACIÓN - CHINÁCOTA"]

In [18]:
temp = secopIps.loc[secopIps.nombre_entidad.isin(ent_eliminar)]
print(f'Se tienen {temp.shape[0]} contratos de entidades que no corresponden a hospitales o E.S.E.')

Se tienen 13854 contratos de entidades que no corresponden a hospitales o E.S.E.


In [19]:
secopIps = secopIps.loc[-secopIps.nombre_entidad.isin(ent_eliminar)]
print(f'Despues de eliminar los contratos de entidades que no son hospitales quedan {secopIps.shape[0]} contratos.')

Despues de eliminar los contratos de entidades que no son hospitales quedan 1391448 contratos.


***
### 8. Quitar el departamento del nombre de la enetidad


> Tarea pendiente

*** 
### 9. Agregar codigo divipola a departamento y municipio de entidad


In [20]:
secopIps.departamento_entidad = secopIps.departamento_entidad.str.upper()

In [21]:
depSecop = pd.DataFrame({'Departamento Secop':secopIps.departamento_entidad.unique()})
depSecop

,Departamento Secop
0,CAUCA
1,BOYACÁ
2,ANTIOQUIA
3,TOLIMA
4,NARIÑO
5,HUILA
6,MAGDALENA
7,BOGOTÁ D.C.
8,CALDAS
9,ATLÁNTICO


In [22]:
# Leer base de datos de departamentos del DANE
departamentosDivipola = pd.read_csv("Datos/cod_divipola_departamento.csv", sep=";")
departamentosDivipola.rename({'Nombre':'Departamento Divipola'}, inplace=True, axis=1)
departamentosDivipola

,Código,Departamento Divipola
0,5,ANTIOQUIA
1,8,ATLÁNTICO
2,11,"BOGOTÁ, D.C."
3,13,BOLÍVAR
4,15,BOYACÁ
5,17,CALDAS
6,18,CAQUETÁ
7,19,CAUCA
8,20,CESAR
9,23,CÓRDOBA


In [23]:
depIntegracion = pd.merge(left=depSecop, right=departamentosDivipola, left_on='Departamento Secop', right_on='Departamento Divipola', how='outer')
depIntegracion

,Departamento Secop,Código,Departamento Divipola
0,CAUCA,19.0,CAUCA
1,BOYACÁ,15.0,BOYACÁ
2,ANTIOQUIA,5.0,ANTIOQUIA
3,TOLIMA,73.0,TOLIMA
4,NARIÑO,52.0,NARIÑO
5,HUILA,41.0,HUILA
6,MAGDALENA,47.0,MAGDALENA
7,BOGOTÁ D.C.,NaN,NaN
8,CALDAS,17.0,CALDAS
9,ATLÁNTICO,8.0,ATLÁNTICO


In [24]:
# Incluir el codigo divipola en el SECOP
secopIps = pd.merge(left=secopIps, right=departamentosDivipola, left_on='departamento_entidad', right_on='Departamento Divipola', how='left')

In [25]:
# Renombrar la columna codigo divipola
secopIps.rename({'Código':'dep_codigo_divipola'}, inplace=True, axis=1)

In [26]:
# Verificar que se integre correctamente el codigo divipola
secopIps.groupby(['departamento_entidad','Departamento Divipola','dep_codigo_divipola']).size()

departamento_entidad  Departamento Divipola  dep_codigo_divipola
AMAZONAS              AMAZONAS               91.0                     1406
ANTIOQUIA             ANTIOQUIA              5.0                    304256
ARAUCA                ARAUCA                 81.0                    31385
ATLÁNTICO             ATLÁNTICO              8.0                     24724
BOLÍVAR               BOLÍVAR                13.0                    16965
BOYACÁ                BOYACÁ                 15.0                    43480
CALDAS                CALDAS                 17.0                    48639
CAQUETÁ               CAQUETÁ                18.0                    18805
CASANARE              CASANARE               85.0                    19435
CAUCA                 CAUCA                  19.0                    26141
CESAR                 CESAR                  20.0                    25335
CHOCÓ                 CHOCÓ                  27.0                      450
CUNDINAMARCA          CUNDINAMARCA 

In [27]:
# Borrar la columna Departamento Divipola
secopIps.drop(['Departamento Divipola'], inplace=True, axis=1)

In [28]:
# Agregar codigo divipola a Bogotá manualmente
secopIps.loc[secopIps.departamento_entidad == 'BOGOTÁ D.C.', 'dep_codigo_divipola'] = 11.0

Ahora vamos a agregar el codigo divipola a los municipios del SECOP I

In [29]:
# Convertir a mayuscula el nombre del municipio
secopIps.municipio_entidad = secopIps.municipio_entidad.str.upper()

In [30]:
munSecop = pd.DataFrame({'Municipio Secop':secopIps.municipio_entidad.unique()})
munSecop

,Municipio Secop
0,SANTANDER DE QUILICHAO
1,UMBITA
2,TARAZÁ
3,FRESNO
4,PASTO
...,...
729,COVEÑAS
730,RÍO VIEJO
731,SIBATÉ
732,SAN FERNANDO


In [31]:
secopIps.columns

Index(['anno_cargue', 'anno_firma', 'nivel_entidad', 'orden_entidad',
       'nombre_entidad', 'nit_entidad', 'tipo_proceso', 'estado_proceso',
       'causa_otras_formas_contr_directa', 'regimen_contratacion',
       'objeto_contratar', 'detalle_objeto_contratar', 'tipo_contrato',
       'municipio_obtencion', 'municipio_entrega', 'municipio_ejecucion',
       'fecha_cargue_secop', 'cuantia_proceso', 'nombre_grupo',
       'nombre_familia', 'nombre_clase', 'tipo_id_contratista',
       'id_contratista', 'razon_social_contratista', 'dpto_mcpo_contratista',
       'tipo_doc_representate_legal', 'id_representante_legal',
       'nombre_representante_legal', 'fecha_firma_contrato',
       'fecha_inicio_ejec_contrato', 'plazo_ejec_contrato',
       'rango_ejec_contrato', 'tiempo_adiciones_dias',
       'tiempo_adiciones_meses', 'fecha_fin_ejec_contrato', 'cuantia_contrato',
       'valor_adiciones', 'valor_total_con_adiciones',
       'objeto_contrato_a_la_firma', 'moneda', 'ruta_web', 'mu

In [32]:
municipiosDivipola = pd.read_csv("Datos/cod_divipola_municipio.csv", sep=";")
municipiosDivipola.rename({'Municipio':'Municipio Divipola'}, inplace=True, axis=1)
municipiosDivipola

,Codigo Departamento,Departamento,Codigo Municipio,Municipio Divipola,Tipo
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio
...,...,...,...,...,...
1116,97,VAUPÉS,97889,YAVARATÉ,Área no municipalizada
1117,99,VICHADA,99001,PUERTO CARREÑO,Municipio
1118,99,VICHADA,99524,LA PRIMAVERA,Municipio
1119,99,VICHADA,99624,SANTA ROSALÍA,Municipio


In [33]:
munSecop = secopIps.groupby(['departamento_entidad','dep_codigo_divipola','municipio_entidad']).size().reset_index().drop(columns=0,index=1)
munSecop

,departamento_entidad,dep_codigo_divipola,municipio_entidad
0,AMAZONAS,91.0,LETICIA
2,ANTIOQUIA,5.0,ALEJANDRÍA
3,ANTIOQUIA,5.0,AMAGÁ
4,ANTIOQUIA,5.0,AMALFI
5,ANTIOQUIA,5.0,ANDES
...,...,...,...
768,VALLE DEL CAUCA,76.0,YOTOCO
769,VALLE DEL CAUCA,76.0,YUMBO
770,VALLE DEL CAUCA,76.0,ZARZAL
771,VAUPÉS,97.0,MITÚ


In [34]:
import fuzzymatcher

In [35]:
left_on = ["departamento_entidad", "municipio_entidad",'dep_codigo_divipola']
right_on = ["Departamento", "Municipio Divipola","Codigo Departamento"]

integracionMun = fuzzymatcher.fuzzy_left_join(munSecop, municipiosDivipola, left_on, right_on)

In [36]:
integracionMun = integracionMun[['best_match_score','__id_left','__id_right','departamento_entidad','Departamento','municipio_entidad','Municipio Divipola','Codigo Departamento','Codigo Municipio']]
integracionMun.index = range(integracionMun.shape[0])
integracionMun

,best_match_score,__id_left,__id_right,departamento_entidad,Departamento,municipio_entidad,Municipio Divipola,Codigo Departamento,Codigo Municipio
0,0.519289,0_left,1088_right,AMAZONAS,AMAZONAS,LETICIA,LETICIA,91,91001
1,0.413738,1_left,3_right,ANTIOQUIA,ANTIOQUIA,ALEJANDRÍA,ALEJANDRÍA,5,5021
2,0.305480,2_left,4_right,ANTIOQUIA,ANTIOQUIA,AMAGÁ,AMAGÁ,5,5030
3,0.305480,3_left,5_right,ANTIOQUIA,ANTIOQUIA,AMALFI,AMALFI,5,5031
4,0.279542,4_left,6_right,ANTIOQUIA,ANTIOQUIA,ANDES,ANDES,5,5034
...,...,...,...,...,...,...,...,...,...
767,0.563663,767_left,1044_right,VALLE DEL CAUCA,VALLE DEL CAUCA,YOTOCO,YOTOCO,76,76890
768,0.583731,768_left,1045_right,VALLE DEL CAUCA,VALLE DEL CAUCA,YUMBO,YUMBO,76,76892
769,0.583731,769_left,1046_right,VALLE DEL CAUCA,VALLE DEL CAUCA,ZARZAL,ZARZAL,76,76895
770,0.414057,770_left,1111_right,VAUPÉS,VAUPÉS,MITÚ,MITÚ,97,97001


Los municipios en los que no realizo el match correctamente:
- MOMPÓS - BOLÍVAR (BOLÍVAR	13468	SANTA CRUZ DE MOMPOX)
- UMBITA - BOYACÁ  (BOYACÁ	15842	ÚMBITA)
- YOLOMBO - ANTIOQUIA (ANTIOQUIA	5890	YOLOMBÓ)
- DON MATÍAS - ANTIOQUIA (ANTIOQUIA	5237	DON MATÍAS)
- SANTA ROSA DE LIMA - BOLÍVAR (BOLÍVAR	13683	SANTA ROSA)
- EL PEÑOL - ANTIOQUIA (ANTIOQUIA	5541	PEÑOL)

Verificar a que municipios corresponde
- SAN MIGUEL DE MOCOA (PUTUMAYO	86001	MOCOA)
- SAN MIGUEL (LA DORADA) (PUTUMAYO	86757	SAN MIGUEL)

In [37]:
repCod = integracionMun.groupby(['Codigo Municipio']).size().reset_index().sort_values(by = 0, ascending=False)
repCod

,Codigo Municipio,0
11,5055,3
760,86757,2
91,5697,2
157,15109,2
0,5001,1
...,...,...
259,17444,1
260,17446,1
261,17486,1
262,17495,1


In [38]:
secopIps.loc[secopIps['municipio_entidad'] == 'YOLOMBO']

,anno_cargue,anno_firma,nivel_entidad,orden_entidad,nombre_entidad,nit_entidad,tipo_proceso,estado_proceso,causa_otras_formas_contr_directa,regimen_contratacion,...,cuantia_contrato,valor_adiciones,valor_total_con_adiciones,objeto_contrato_a_la_firma,moneda,ruta_web,municipio_entidad,departamento_entidad,nombre_entidad_mod,dep_codigo_divipola
5,2018,2018.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,71400,0,71400,SERVICIO DE MANTENIMIENTO PARA AMBULANCIA,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
286,2019,2019.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,890720,0,890720,COMPRA DE MATERIAL DE LABORATORIO,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
422,2019,2019.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,135000,0,135000,COMPRA MATERIAL DE LABORATORIO,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
475,2019,2019.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,7007196,0,7007196,COMPRA DE PAPLERIA DE OFICINA,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
634,2018,2018.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,965000,0,965000,COMPRA DE MATERIAL DE ORTOPEDIA,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389220,2018,2018.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,119000,0,119000,COMPRA MEDICAMENTOS,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
1389941,2019,2019.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,1114200,0,1114200,COMPRA MEDICAMENTOS,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
1390255,2018,2018.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,2683550,0,2683550,COMPRA MATERIAL ORTOPEDIA,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0
1390580,2018,2018.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN RAFAEL - YOLOMBÓ,890981536,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,2290925,0,2290925,COMPRA REGRIGERIOS PARA SEMANA DE LACTANCIA MA...,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,YOLOMBO,ANTIOQUIA,antioquia ese hospital san rafael yolombó,5.0


In [39]:
secopIps.loc[secopIps['departamento_entidad'] == 'PUTUMAYO']['municipio_entidad'].unique()

array(['VALLE DEL GUAMUEZ/LA HORMIGA', 'SAN MIGUEL DE MOCOA',
       'PUERTO LEGUÍZAMO', 'PUERTO CAICEDO', 'PUERTO ASÍS', 'ORITO',
       'SAN MIGUEL (LA DORADA)', 'PUERTO GUZMÁN', 'COLÓN'], dtype=object)

In [40]:
integracionMun.loc[integracionMun['municipio_entidad'] == 'SAN MIGUEL (LA DORADA)']

,best_match_score,__id_left,__id_right,departamento_entidad,Departamento,municipio_entidad,Municipio Divipola,Codigo Departamento,Codigo Municipio
563,0.299358,563_left,1082_right,PUTUMAYO,PUTUMAYO,SAN MIGUEL (LA DORADA),SAN MIGUEL,86,86757


In [41]:
integracionMun.loc[integracionMun['Codigo Municipio'] == 15109]

,best_match_score,__id_left,__id_right,departamento_entidad,Departamento,municipio_entidad,Municipio Divipola,Codigo Departamento,Codigo Municipio
163,0.259582,163_left,205_right,BOYACÁ,BOYACÁ,BUENAVISTA,BUENAVISTA,15,15109
250,-0.093910,250_left,205_right,BOYACÁ,BOYACÁ,UMBITA,BUENAVISTA,15,15109


In [42]:
# Buscar municipios por nombre
municipiosDivipola.loc[municipiosDivipola["Municipio Divipola"].str.contains('SAN MIGUEL')]

,Codigo Departamento,Departamento,Codigo Municipio,Municipio Divipola,Tipo
280,15,BOYACÁ,15676,SAN MIGUEL DE SEMA,Municipio
917,68,SANTANDER,68686,SAN MIGUEL,Municipio
1082,86,PUTUMAYO,86757,SAN MIGUEL,Municipio


In [43]:
# Buscar municipios por departamento
municipiosDivipola.loc[municipiosDivipola['Departamento'] == 'BOLIVAR']['Municipio Divipola'].unique()

array([], dtype=object)

In [44]:
# Ajustar algunos municipios manualmente
municipioSecop = ['MOMPÓS','UMBITA','YOLOMBO','DON MATÍAS','SANTA ROSA DE LIMA','EL PEÑOL','SAN MIGUEL (LA DORADA)','SAN MIGUEL DE MOCOA']
#municipioDane = ['SANTA CRUZ DE MOMPOX','ÚMBITA','YOLOMBÓ','DON MATÍAS','SANTA ROSA','PEÑOL','SAN MIGUEL','MOCOA']
departamentoSecop = ['BOLÍVAR','BOYACÁ','ANTIOQUIA','ANTIOQUIA','BOLÍVAR','ANTIOQUIA','PUTUMAYO','PUTUMAYO']
codMunDivipola = [13468,15842,5890,5237,13683,5541,86757,86001]
k = 0
for (i,j) in zip(municipioSecop, departamentoSecop):
    integracionMun.loc[(integracionMun.municipio_entidad == i) & (integracionMun.departamento_entidad == j), 'Codigo Municipio'] = codMunDivipola[k]
    k+=1

In [45]:
# Verificar la integración correcta de los codigos de los municipios
for (i,j) in zip(municipioSecop, departamentoSecop):
    print(integracionMun.loc[(integracionMun.municipio_entidad == i) & (integracionMun.departamento_entidad == j), ['departamento_entidad','municipio_entidad','Codigo Municipio']])

    departamento_entidad municipio_entidad  Codigo Municipio
144              BOLÍVAR            MOMPÓS             13468
    departamento_entidad municipio_entidad  Codigo Municipio
250               BOYACÁ            UMBITA             15842
    departamento_entidad municipio_entidad  Codigo Municipio
112            ANTIOQUIA           YOLOMBO              5890
   departamento_entidad municipio_entidad  Codigo Municipio
37            ANTIOQUIA        DON MATÍAS              5237
    departamento_entidad   municipio_entidad  Codigo Municipio
152              BOLÍVAR  SANTA ROSA DE LIMA             13683
   departamento_entidad municipio_entidad  Codigo Municipio
41            ANTIOQUIA          EL PEÑOL              5541
    departamento_entidad       municipio_entidad  Codigo Municipio
563             PUTUMAYO  SAN MIGUEL (LA DORADA)             86757
    departamento_entidad    municipio_entidad  Codigo Municipio
564             PUTUMAYO  SAN MIGUEL DE MOCOA             86001


In [46]:
integracionMun.drop(columns=['best_match_score','__id_left','__id_right','Departamento','Municipio Divipola','Codigo Departamento'],
                    inplace=True)

In [47]:
integracionMun

,departamento_entidad,municipio_entidad,Codigo Municipio
0,AMAZONAS,LETICIA,91001
1,ANTIOQUIA,ALEJANDRÍA,5021
2,ANTIOQUIA,AMAGÁ,5030
3,ANTIOQUIA,AMALFI,5031
4,ANTIOQUIA,ANDES,5034
...,...,...,...
767,VALLE DEL CAUCA,YOTOCO,76890
768,VALLE DEL CAUCA,YUMBO,76892
769,VALLE DEL CAUCA,ZARZAL,76895
770,VAUPÉS,MITÚ,97001


In [48]:
secopIps = pd.merge(left=secopIps, right=integracionMun, on=['departamento_entidad','municipio_entidad'], how='left')

In [49]:
secopIps.rename(columns={'Codigo Municipio':'municipio_codigo_divipola'}, inplace=True)

In [50]:
secopIps['dep_codigo_divipola'] = np.int64(secopIps['dep_codigo_divipola']*1000)
secopIps['municipio_codigo_divipola'] = np.int64(secopIps['municipio_codigo_divipola'])

In [51]:
secopIps.head()

,anno_cargue,anno_firma,nivel_entidad,orden_entidad,nombre_entidad,nit_entidad,tipo_proceso,estado_proceso,causa_otras_formas_contr_directa,regimen_contratacion,...,valor_adiciones,valor_total_con_adiciones,objeto_contrato_a_la_firma,moneda,ruta_web,municipio_entidad,departamento_entidad,nombre_entidad_mod,dep_codigo_divipola,municipio_codigo_divipola
0,2015,2015.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,CAUCA - E.S.E. HOSPITAL FRANCISCO DE PAULA SAN...,891500084,Régimen Especial,Liquidado,Ninguna,Régimen Especial,...,0,44000000,PRESTAR LOS SERVICIOS MÉDICOS ESPECIALIZADOS E...,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,SANTANDER DE QUILICHAO,CAUCA,cauca ese hospital francisco de paula santand...,19000,19698
1,2019,2018.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,BOYACÁ - E.S.E. CENTRO DE SALUD SAN RAFAEL,820003782,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,0,3558100,SUMINISTRO DE EQUIPO DE COMPUTO PORTÁTIL PARA ...,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,UMBITA,BOYACÁ,boyacá ese centro de salud san rafael,15000,15842
2,2017,2017.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,ANTIOQUIA - E.S.E. HOSPITAL SAN ANTONIO - TARAZÁ,890984696,Contratación Directa (Ley 1150 de 2007),Celebrado,Prestación de Servicios Profesionales y de Apo...,Estatuto General de Contratación,...,0,18700000,La contratista se compromete para con la E.S.E...,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,TARAZÁ,ANTIOQUIA,antioquia ese hospital san antonio tarazá,5000,5790
3,2017,2016.0,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,TOLIMA - HOSPITAL SAN VICENTE DE PAÚL - FRESNO,890700901,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,0,18000000,SE REQUIERE CONTRATAR EL SERVICIO DE PERSONAL...,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,FRESNO,TOLIMA,tolima hospital san vicente de paúl fresno,73000,73283
4,2016,2016.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 1,NARIÑO - E.S.E. CENTRO DE SALUD SAN MIGUEL ARC...,900126676,Régimen Especial,Celebrado,Ninguna,Régimen Especial,...,0,689454,servicios asistenciales como Auxiliar en salud...,Pesos (COP),https://www.contratos.gov.co/consultas/detalle...,PASTO,NARIÑO,nariño ese centro de salud san miguel arcangel,52000,52001


In [52]:
# integracionMun.to_csv('Datos/integracionMun.csv')

***
### 10. Copiar datos en archivo de datos

In [53]:
#compression_opts = dict(method='gzip', archive_name='secop_ips_i_limpia.csv')  
secopIps.to_csv('Datos/secop_ips_i_limpia.gz', index=False)  